# Setting Up The SQLite Server
Below is the code to import the sqlite3 library that is included with the Python standard library package.

In [1]:
import sqlite3

Next is the code to create/connect with the database

In [2]:
connection = sqlite3.connect('movies.db')

Now the next code will create a cursor object so we can execute SQL statements

In [3]:
cursor = connection.cursor()

## Drop Tables
Drop all the tables from the database to start fresh.

In [ ]:
cursor.execute('DROP TABLE IF EXISTS movie_stars')
cursor.execute('DROP TABLE IF EXISTS stars')
cursor.execute('DROP TABLE IF EXISTS movie_genres')
cursor.execute('DROP TABLE IF EXISTS genres')
cursor.execute('DROP TABLE IF EXISTS movie_directors')
cursor.execute('DROP TABLE IF EXISTS directors')
cursor.execute('DROP TABLE IF EXISTS imdb_movies')
cursor.execute('DROP TABLE IF EXISTS certifications')

## Create Tables 

### Main Table From imdb dataset (imdb_movies)

In [ ]:
cursor.execute('''CREATE TABLE IF NOT EXISTS imdb_movies
               (
                    imdb_movies_id INTEGER PRIMARY KEY NOT NULL, 
                    movie_name VARCHAR(100) NOT NULL, 
                    year_released TEXT NOT NULL,
                    run_time INTEGER NOT NULL,
                    meta_score INTEGER,
                    description TEXT NOT NULL,
                    certification VARCHAR(25)
                )''')

### Create table for the stars

In [ ]:
cursor.execute('''CREATE TABLE IF NOT EXISTS stars
               (
                    star_id INTEGER PRIMARY KEY NOT NULL, 
                    star_name VARCHAR(100) 
                )''')

### Make table that connects movies and stars
We are connecting movies and stars in a separate table because there can be multiple stars per movie and each star can be in multiple movies. Since there is a many-to-many relationship between the movies and stars tables, we need to make a new table that contains the id from the movies table and the id from the stars table. This will also be the same with movie directors and genres.

In [ ]:
cursor.execute('''CREATE TABLE IF NOT EXISTS movie_stars
               (
                    movie_star_id INTEGER PRIMARY KEY AUTOINCREMENT, 
                    movie_id INTEGER NOT NULL,
                    star_id INTEGER NOT NULL,
                    FOREIGN KEY (movie_id) REFERENCES imdb_movies(imdb_movies_id)
                    FOREIGN KEY (star_id) REFERENCES stars(star_id)
                )''')

### Make table for Directors

In [ ]:
cursor.execute('''CREATE TABLE IF NOT EXISTS directors
               (
                    director_id INTEGER PRIMARY KEY NOT NULL, 
                    director_name VARCHAR(100) 
                )''')

### Make a table that connects Movies and Directors
We need to create a table that connects movies and directors like we did with movies and stars

In [ ]:
cursor.execute('''CREATE TABLE IF NOT EXISTS movie_directors
               (
                    movie_director_id INTEGER PRIMARY KEY AUTOINCREMENT, 
                    movie_id INTEGER NOT NULL,
                    director_id INTEGER NOT NULL,
                    FOREIGN KEY (movie_id) REFERENCES imdb_movies(imdb_movies_id)
                    FOREIGN KEY (director_id) REFERENCES directors(director_id)
                )''')

### Create table containing all the different genres

In [ ]:
cursor.execute('''CREATE TABLE IF NOT EXISTS genres
               (
                    genre_id INTEGER PRIMARY KEY NOT NULL, 
                    genre_name VARCHAR(20) 
                )''')

### Make a table connecting movies and genres
Like we did with movies and stars, and movies and directors, we will make a table connecting the movie values with the genre values.

In [ ]:
cursor.execute('''CREATE TABLE IF NOT EXISTS movie_genres
               (
                    movie_genre_id INTEGER PRIMARY KEY AUTOINCREMENT, 
                    movie_id INTEGER NOT NULL,
                    genre_id INTEGER NOT NULL,
                    FOREIGN KEY (movie_id) REFERENCES imdb_movies(imdb_movies_id)
                    FOREIGN KEY (genre_id) REFERENCES genres(genre_id)
                )''')

## Use pandas to clean up our data
Using pandas we will clean up our data to only have movies released from 2018 to 2022

In [ ]:
# import pandas package
import pandas as pd

# path to our movie data file
path_to_data = './datasets/imdb/imdb-data.csv'

# open the csv file
imdb_df = pd.read_csv(path_to_data, index_col=0)

movies_afer_2018 = imdb_df["Year of Release"] >= 2018
movies_before_2023 = imdb_df["Year of Release"] <= 2022

movies_in_range_df = imdb_df[movies_afer_2018 & movies_before_2023]
movies_in_range_df = movies_in_range_df.sort_values(by="Year of Release")

movies_in_range_df

In [ ]:
import ast

genre_s = movies_in_range_df["Genre"]

genre_list = []

for genres in genre_s:
    lit_genres = ast.literal_eval(genres)
    for genre in lit_genres:
        genre_list.append(genre.strip())

genre_set = set(genre_list)

for genre in genre_set:
    print(genre)

#### Adding Genres to the genres table

In [ ]:
id = 1

for genre in genre_set:
    cursor.execute('INSERT INTO genres (genre_id, genre_name) VALUES (?, ?)', (id, genre))
    id += 1

In [ ]:
cursor.execute('SELECT * FROM genres')
rows = cursor.fetchall()
for row in rows:
    print(row)

## Get All the Directors
Get all the directors from the DataFrame like was done with the genres and insert them into the directors table.

In [ ]:
director_s = movies_in_range_df["Director"]

director_list = []

for director in director_s:
    lit_director = ast.literal_eval(director)
    for each in lit_director:
        director_list.append(each.strip())


director_set = set(director_list)

print(director_set)

### Adding Directors to the Directors Table

In [ ]:
id = 1

for director in director_set:
    cursor.execute('INSERT INTO directors (director_id, director_name) VALUES (?, ?)', (id, director))
    id += 1

In [ ]:
cursor.execute('SELECT * FROM directors')
rows = cursor.fetchall()
for row in rows:
    print(row)

## Get all the Stars
Get all the stars from the DataFrame to insert into the stars table

In [ ]:
star_s = movies_in_range_df["Stars"]

stars_list = []

for star in star_s:
    lit_star = ast.literal_eval(star)
    for each in lit_star:
        stars_list.append(each.strip())


stars_set = set(stars_list)

print(stars_set)

#### Adding stars to the stars table

In [ ]:
id = 1

for star in stars_set:
    cursor.execute('INSERT INTO stars (star_id, star_name) VALUES (?, ?)', (id, star))
    id += 1

In [ ]:
cursor.execute('SELECT * FROM stars')
rows = cursor.fetchall()
for row in rows:
    print(row)

### Add the Movies to the imdb_movies Table
Now we will add all the Movies and the proper data to the imdb_movies table

In [ ]:
import re

for index, row in movies_in_range_df.iterrows():
    # Declare a variable for each field and assign it to value from DF
    movie_id = index
    movie_name = re.sub(r'[^\w\s]', '',row["Movie Name"]).upper()
    year_released = row["Year of Release"]
    run_time = row["Run Time in minutes"]
    meta_score = row["MetaScore"]
    description = " ".join(ast.literal_eval(row["Description"]))
    certification = row["Certification"]
    print(f"{movie_id}, {movie_name}, {year_released}, {run_time}, {meta_score}, {description}, {certification}")
    cursor.execute('''INSERT INTO imdb_movies (imdb_movies_id, movie_name, year_released, run_time, meta_score, description, certification) 
                      VALUES (?, ?, ?, ?, ?, ?, ?)''', (movie_id, movie_name, year_released, run_time, meta_score, description, certification))

    # Make movie names with all caps and no special characters to match with other datasets
    # Use INSERT statement to add values to the database

In [ ]:
cursor.execute('SELECT * FROM imdb_movies')
rows = cursor.fetchall()
for row in rows:
    print(row)

## Get IDs from each movie and Directors to insert into movie_directors table
Loop through the pandas dataframe holding all of the data, get the ID of the movie and the ID of the director and insert into the movie_directors. Will have to loop through the directors since it is a list inside of the DataFrame and individually get the id of each from the directors table. 

In [ ]:
movie_director_index = 1

for index, row in movies_in_range_df.iterrows():
    movie_id = index
    directors = ast.literal_eval(row["Director"])
    for director in directors:
        cursor.execute("SELECT director_id FROM directors WHERE director_name = ?", (director.strip(),))
        dir_ids = cursor.fetchall()
        for dir_id in dir_ids:
            cursor.execute("INSERT INTO movie_directors VALUES (?,?,?)", (movie_director_index, movie_id, dir_id[0]))
            movie_director_index += 1
        

In [ ]:
cursor.execute("SELECT * FROM movie_directors")
rows = cursor.fetchall()

for row in rows:
    print(row)

## Get IDs from Movies and Stars to insert into movie_stars Table
Loop through the movies table and stars table to get their ids and then insert them into a new row in the movie_stars table.

In [ ]:
movie_star_index = 1

for index, row in movies_in_range_df.iterrows():
    movie_id = index
    stars = ast.literal_eval(row["Stars"])
    for star in stars:
        cursor.execute("SELECT star_id FROM stars WHERE star_name = ?", (star.strip(),))
        star_ids = cursor.fetchall()
        for star_id in star_ids:
            cursor.execute("INSERT INTO movie_stars VALUES (?,?,?)", (movie_star_index, movie_id, star_id[0]))
            movie_star_index += 1

In [ ]:
cursor.execute("SELECT * FROM movie_stars")
rows = cursor.fetchall()

for row in rows:
    print(row)

## Get IDs from Movies and Genres and Place them into the movie_genres table
Get the id from the imdb_movies table and each genre and place them in rows in the movie_genres table

In [ ]:
movie_genre_index = 1

for index, row in movies_in_range_df.iterrows():
    movie_id = index
    genres = ast.literal_eval(row["Genre"])
    for genre in genres:
        cursor.execute("SELECT genre_id FROM genres WHERE genre_name = ?", (genre.strip(),))
        genre_ids = cursor.fetchall()
        #print(genre_ids)
        for genre_id in genre_ids:
            cursor.execute("INSERT INTO movie_genres VALUES (?,?,?)", (movie_genre_index, movie_id, genre_id[0]))
            movie_genre_index += 1

In [ ]:
cursor.execute("SELECT * FROM movie_genres")
rows = cursor.fetchall()

for row in rows:
    print(row)